In [1]:
import pymongo
import time
import threading

In [2]:
cl = pymongo.MongoClient("mongodb://localhost:27017")
db = cl["deep"]
col = db["flat"]

In [3]:
col.find_one()

{'_id': ObjectId('6866846b7fb3cacee8f2e148'),
 'number_of_records': 600,
 'activity_sec': 38,
 'application': 'floor',
 'device': 2346,
 'volume_total_bytes': 5855,
 'subscribers': 709}

In [4]:
col.count_documents({"device": {"$gte" : 9000}})

50159

In [21]:
indexes = col.list_indexes()
for i in indexes:
    print(i)

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('device', 1)])), ('name', 'device_1')])


In [6]:
def calc_perf(old, new):
    rounded_old = round(old)
    rounded_new = round(new)
    ratio = ((rounded_old - rounded_new) / rounded_old) * 100
    print(f"{rounded_old} {rounded_new} {ratio:.1f}")

In [18]:
group_pipeline = [
    {"$group": {"_id": "$device"}},
    {"$sort": {"_id":1}}
]

avg_pipeline = [
    {"$match": {"device": {"$gte": 9000}}},
    {"$group": {"_id": None, "average_volume": {"$avg": "$volume_total_bytes"}}}
]

In [8]:
def measure_group(user_id, duration_list):
    duration = 0
    for i in range(10):
        start = time.perf_counter()
        result = list(col.aggregate(group_pipeline))
        end = time.perf_counter()
        duration += end - start
    #take average, convert to ms
    duration = (duration / 10) * 1000
    duration_list[user_id] = duration

In [9]:
def group_test(num_of_users):
    threads_list = []
    duration_list = [None] * num_of_users
    
    for x in range(num_of_users):
        t = threading.Thread(target=measure_group, args=(x, duration_list, ))
        t.start()
        threads_list.append(t)

    for t in threads_list:
        t.join()

    result = sum(duration_list) / len(duration_list)
    return result

In [10]:
def measure_average(user_id, duration_list):
    duration = 0
    avg_vol = 0
    for i in range(10):
        start = time.perf_counter()
        execution = next(col.aggregate(avg_pipeline), None)
        end = time.perf_counter()
        duration += end - start

    duration = (duration / 10) * 1000
    duration_list[user_id] = duration

In [11]:
def average_test(num_of_users):
    threads_list = []
    duration_list = [None] * num_of_users

    for x in range(num_of_users):
        t = threading.Thread(target=measure_average, args=(x, duration_list))
        t.start()
        threads_list.append(t)

    for t in threads_list:
        t.join()
    result = sum(duration_list) / len(duration_list)
    return result

In [19]:
group_one_old = group_test(1)
group_two_old = group_test(2)
group_four_old = group_test(4)

average_one_old = average_test(1)
average_two_old = average_test(2)
average_four_old = average_test(4)

In [14]:
col.create_index([("device", 1)])

'device_1'

In [20]:
group_one_new = group_test(1)
group_two_new = group_test(2)
group_four_new = group_test(4)

average_one_new = average_test(1)
average_two_new = average_test(2)
average_four_new = average_test(4)

In [22]:
calc_perf(group_one_old, group_one_new)
calc_perf(group_two_old, group_two_new)
calc_perf(group_four_old, group_four_new)

50 55 -10.0
52 53 -1.9
84 84 0.0


In [17]:
calc_perf(average_one_old, average_one_new)
calc_perf(average_two_old, average_two_new)
calc_perf(average_four_old, average_four_new)

192 96 50.0
196 98 50.0
411 166 59.6
